# Wasserstein barycenters - Demo Notebook

In [ ]:
import numpy as np
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from wasserstein_barycenters.wasserstein_barycenters_3d import convolutional_barycenter_3d
from wasserstein_barycenters.utils import point_cloud_to_mesh, plot_mesh

## Inital shapes

First, let's import some shapes that are represented as point clouds

In [ ]:
shape1 = np.load('data/numpy/duck.npy')
shape2 = np.load('data/numpy/torus.npy')
initial_shapes = np.concatenate((shape1[np.newaxis, :, :, :], shape2[np.newaxis, :, :, :]), axis=0)

Second, let's figure out what is the dimension of the arrays so we can initialise what we call "space" that is the discrete support of the indicative functions of the interior of the shapes 

In [ ]:
N = shape1.shape[0]
t = np.linspace(0, 1, N)
space = np.stack(np.meshgrid(t, t, t), axis=-1).reshape(N**3, -1)

Now we can plot our inital shapes 

In [ ]:
fig = make_subplots(rows=1, cols=2, specs=[[{"type": "scatter3d"}, {"type": "scatter3d"}]])
for i in range(2):
    shape = initial_shapes[i].astype(bool)
    vertices, faces = point_cloud_to_mesh(shape, space)
    fig.add_trace(
        go.Mesh3d(
                x=vertices[:, 0],
                y=vertices[:, 1],
                z=vertices[:, 2],
                i=faces[:, 0],
                j=faces[:, 1],
                k=faces[:, 2],
            ),
        row=1, col=i+1
    )
fig.update_layout(height=400, width=800)
fig.update_scenes(
            xaxis_visible=False, yaxis_visible=False, zaxis_visible=False
        )
fig.show()

## Wasserstein barycenters

First, let's redefine our point clouds to distributions by just dividing them by their volume.

In [ ]:
# points clouds to distributions 
initial_shapes = initial_shapes / np.sum(initial_shapes, axis=(1, 2, 3), keepdims=True)

Now, we would like to have intermediary shapes between these two. We are going to use an interpolation based on Wasserstein distance. First, let us define weights, that will be the contribution of each of the inital shapes to the final result.  

In [ ]:
n_steps = 5
t = np.linspace(0,1,n_steps)
weights = np.array([[s,1-s] for s in t])
print("Weights for the barycenters:\n", weights)

Now let's compute our barycenters using `convolutional_barycenter_3d`. Then we rescale it by dividing it by its maximum value.

In [ ]:
barycenters = []

for alpha in weights:
    barycenter = convolutional_barycenter_3d(initial_shapes, alpha)
    barycenter = barycenter/max(barycenter.flatten())
    barycenters.append(barycenter)

According to our studied paper, we define the indicative functions of the interior of the barycenters by extracting the level set corresponding to the half the maximum probability value. We reconstruct the surface of our point clouds and convert it to a mesh, so we can use plotly feature of lighting of the shape to explore it better. You can select any shape you want by modifying `shape_index` with a value between 0 and q.

In [ ]:
shape_index = 2
cloud = barycenters[shape_index] > 0.5
vertices, faces = point_cloud_to_mesh(cloud, space)
plot_mesh(vertices, faces, show_axis=False)

Note : this code can be used to mix more than 2 shapes !